> https://www.ectimes.org.tw/1998/

In [25]:
import requests
from bs4 import BeautifulSoup

url = "https://www.ectimes.org.tw/2025/"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
response.encoding = "utf-8"

soup = BeautifulSoup(response.text, "html.parser")

# 使用 CSS selector
links = soup.select("#main > div > div > div > header > h2 > a")

# 存成 txt
with open("links.csv", "w", encoding="utf-8") as f:
    f.write("title,href\n")
    for a in links:
        href = a.get("href")
        title = a.get_text(strip=',')

        if href:
            f.write(f"{title},{href}\n")

print(f"成功抓到 {len(links)} 筆連結，已存成 links.csv")


成功抓到 12 筆連結，已存成 links.csv


In [ ]:
# import requests, csv
# from bs4 import BeautifulSoup

# for i in range(0, 26):
#     for j in range(2,61):
#         url = f"https://www.ectimes.org.tw/{2000+i}/page/{j}/"

#         response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
#         response.encoding = "utf-8"

#         soup = BeautifulSoup(response.text, "html.parser")
#         links = soup.select("#main > div > div > div > header > h2 > a")

#         with open("links.csv", "a", newline="", encoding="utf-8") as f:
#             writer = csv.writer(f)
#             for a in links:
#                 writer.writerow([
#                     a.get_text(strip=True),
#                     a.get("href")
#                 ])


In [1]:
import requests, csv
from bs4 import BeautifulSoup

for i in range(0, 26):
    for j in range(61,100):
        url = f"https://www.ectimes.org.tw/{2000+i}/page/{j}/"
        print(f"\r目前進度: {2000+i}第{j}頁",end="")
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            response.encoding = "utf-8"

            soup = BeautifulSoup(response.text, "html.parser")
            links = soup.select("#main > div > div > div > header > h2 > a")

            with open("links_new.csv", "a", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                for a in links:
                    writer.writerow([
                        a.get_text(strip=True),
                        a.get("href")
                    ])
        except:
            print(f"\n找不到{2000+i}第{j}頁資料",end="")
            break


目前進度: 2025第99頁

In [2]:
import requests
from bs4 import BeautifulSoup
import re

def safe_filename(name):
    return re.sub(r'[\\/:*?"<>|\[\]]', '_', name)

headers = {
    "User-Agent": "Mozilla/5.0"
}

import pandas as pd
link = pd.read_csv("links_new.csv")

j = 0

for i in range(2222,len(link)):
    title = link.iloc[i,0]
    href = link.iloc[i,1]
    # request link
    response = requests.get(href, headers=headers)
    response.encoding = "utf-8"

    soup = BeautifulSoup(response.text, "html.parser")
    # date obtain
    date = soup.select_one("#main > div > header > div > div.date > a")
    data_text = date.get_text(strip=True)
    filename = safe_filename(data_text)
    # 找到 entry-content
    entry = soup.select_one("#main > div > div.entry-content")

    if entry:
        paragraphs = entry.find_all("p")

        with open(f"docs/{filename}.txt", "w", encoding="utf-8") as f:
            f.write(f"標題: {title}\n原文: {href}\n")
            for p in paragraphs:
                text = p.get_text(strip=True)
                if text:  # 避免空段落
                    f.write(text + "\n\n")
        print(f"\r進度: {i}/{len(link)}={round(i/len(link),4)*100}%", end='')
        # print(f"成功抓到 {len(paragraphs)} 段內容\r")
    else:
        print("找不到 entry-content")

進度: 3893/3894=99.97%00000000001%%